In [ ]:
from transformer import Transformer # this is the transformer.py file
import torch
import numpy as np

In [ ]:
english_file = 'train.en' 
mgl_file = 'train.mn' 

START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''

mgl_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '₮', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ', 
                      'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 
                      'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'ө', 'п', 'р', 'с', 'т', 'у', 'ү', 
                      'ф', 'х', 'ц', 'ч', 'ш', 
                      'щ', 'ъ', 'ь', 'ы', 'э', 
                      'ю', 'я', PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '₮', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ', 
                      'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 
                      'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'w', 'x', 'y', 
                      'z', PADDING_TOKEN, END_TOKEN]

In [ ]:
index_to_mgl = {k:v for k,v in enumerate(mgl_vocabulary)}
mgl_to_index = {v:k for k,v in enumerate(mgl_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [ ]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(mgl_file, 'r') as file:
    mgl_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
mgl_sentences = mgl_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
mgl_sentences = [sentence.rstrip('\n') for sentence in mgl_sentences]

In [ ]:
english_sentences[:10]

In [ ]:
mgl_sentences[:10]

In [ ]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length mgl: {np.percentile([len(x) for x in mgl_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )

In [ ]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(mgl_sentences)):
    mgl_sentence, english_sentence = mgl_sentences[index], english_sentences[index]
    if is_valid_length(mgl_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(mgl_sentence, mgl_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(mgl_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

In [ ]:
mgl_sentences = [mgl_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [ ]:
mgl_sentences[:3]

In [ ]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
mn_vocab_size = len(mgl_vocabulary)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          mn_vocab_size,
                          english_to_index,
                          mgl_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [ ]:
transformer

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, mgl_sentences):
        self.english_sentences = english_sentences
        self.mgl_sentences = mgl_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.mgl_sentences[idx]

In [ ]:
dataset = TextDataset(english_sentences, mgl_sentences)

In [ ]:
len(dataset)

In [ ]:
dataset[1]

In [ ]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [ ]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

In [ ]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=mgl_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
NEG_INFTY = -1e9

def create_masks(eng_batch, mn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, mn_sentence_length = len(eng_batch[idx]), len(mn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      mn_chars_to_padding_mask = np.arange(mn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, mn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, mn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, mn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [ ]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, mn_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, mn_batch)
        optim.zero_grad()
        mn_predictions = transformer(eng_batch,
                                     mn_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(mn_batch, start_token=False, end_token=True)
        loss = criterian(
            mn_predictions.view(-1, mn_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == mgl_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"mgl Translation: {mn_batch[0]}")
            mn_sentence_predicted = torch.argmax(mn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in mn_sentence_predicted:
              if idx == mgl_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_mgl[idx.item()]
            print(f"mgl Prediction: {predicted_sentence}")


            transformer.eval()
            mn_sentence = ("",)
            eng_sentence = ("бн",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, mn_sentence)
                predictions = transformer(eng_sentence,
                                          mn_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_mgl[next_token_index]
                mn_sentence = (mn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (should we go to the mall?) : {mn_sentence}")
            print("-------------------------------------------")

In [ ]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  mn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, mn_sentence)
    predictions = transformer(eng_sentence,
                              mn_sentence,
                              encoder_self_attention_mask.to(device), 
                              decoder_self_attention_mask.to(device), 
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_mgl[next_token_index]
    mn_sentence = (mn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return mn_sentence[0]

In [ ]:
translation = translate("what should we do when the day starts?")
print(translation)
